Loading Libraries

In [1]:
import pandas as pd
import numpy as np
from dask import dataframe as dd
from dask.distributed import Client

Setting up paths to csv files / datasets

In [2]:
# CSV-01-12
path_DdoS_DNS = "D:\\thesis_dataset\\01-12\\DrDoS_DNS.csv"
path_DdoS_MSSQL = "D:\\thesis_dataset\\01-12\\DrDoS_MSSQL.csv"
path_DdoS_LDAP = "D:\\thesis_dataset\\01-12\\DrDoS_LDAP.csv"
path_DdoS_NTP = "D:\\thesis_dataset\\01-12\\DrDoS_NTP.csv"
path_DdoS_NetBIOS = "D:\\thesis_dataset\\01-12\\DrDoS_NetBIOS.csv"
path_DdoS_SNMP = "D:\\thesis_dataset\\01-12\\DrDoS_SNMP.csv"
path_DdoS_SSDP = "D:\\thesis_dataset\\01-12\\DrDoS_SSDP.csv"
path_DdoS_UDP = "D:\\thesis_dataset\\01-12\\DrDoS_UDP.csv"
path_Syn = "D:\\thesis_dataset\\01-12\\Syn.csv"
path_TFTP = "D:\\thesis_dataset\\01-12\\TFTP.csv"
path_UDPLag = "D:\\thesis_dataset\\01-12\\UDPLag.csv"

# # CSV-03-11
# path__LDAP = "../CICDDoS-2019/CSV-03-11/03-11/LDAP.csv"
# path__MSSQL = "../CICDDoS-2019/CSV-03-11/03-11/MSSQL.csv"
# path__NetBIOS = "../CICDDoS-2019/CSV-03-11/03-11/NetBIOS.csv"
# path__Portmap = "../CICDDoS-2019/CSV-03-11/03-11/Portmap.csv"
# path__Syn = "../CICDDoS-2019/CSV-03-11/03-11/Syn.csv"
# path__UDP = "../CICDDoS-2019/CSV-03-11/03-11/UDP.csv"
# path__UDPLag = "../CICDDoS-2019/CSV-03-11/03-11/UDPLag.csv"

paths = [path_DdoS_DNS, path_DdoS_MSSQL, path_DdoS_LDAP, path_DdoS_NTP, path_DdoS_NetBIOS, path_DdoS_SNMP, path_DdoS_SSDP, path_DdoS_UDP, path_Syn, path_TFTP, path_UDPLag]
# , path__LDAP, path__MSSQL, path__NetBIOS,
#      path__Portmap, path__Syn, path__UDP, path__UDPLag]

Column / feature names

In [3]:
col_name_consistency = {
    'Flow ID': 'Flow ID',
    'Source IP': 'Source IP',
    'Src IP': 'Source IP',
    'Source Port': 'Source Port',
    'Src Port': 'Source Port',
    'Destination IP': 'Destination IP',
    'Dst IP': 'Destination IP',
    'Destination Port': 'Destination Port',
    'Dst Port': 'Destination Port',
    'Protocol': 'Protocol',
    'Timestamp': 'Timestamp',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Total Fwd Packets',
    'Tot Fwd Pkts': 'Total Fwd Packets',
    'Total Backward Packets': 'Total Backward Packets',
    'Tot Bwd Pkts': 'Total Backward Packets',
    'Total Length of Fwd Packets': 'Fwd Packets Length Total',
    'TotLen Fwd Pkts': 'Fwd Packets Length Total',
    'Total Length of Bwd Packets': 'Bwd Packets Length Total',
    'TotLen Bwd Pkts': 'Bwd Packets Length Total',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Pkt Len Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Pkt Len Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Pkt Len Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Fwd Pkt Len Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Pkt Len Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Pkt Len Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Pkt Len Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Bwd Pkt Len Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Byts/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow Pkts/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Tot': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Tot': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    'Bwd PSH Flags': 'Bwd PSH Flags',
    'Fwd URG Flags': 'Fwd URG Flags',
    'Bwd URG Flags': 'Bwd URG Flags',
    'Fwd Header Length': 'Fwd Header Length',
    'Fwd Header Len': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Bwd Header Len': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Fwd Pkts/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Bwd Pkts/s': 'Bwd Packets/s',
    'Min Packet Length': 'Packet Length Min',
    'Pkt Len Min': 'Packet Length Min',
    'Max Packet Length': 'Packet Length Max',
    'Pkt Len Max': 'Packet Length Max',
    'Packet Length Mean': 'Packet Length Mean',
    'Pkt Len Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Pkt Len Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'Pkt Len Var': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'FIN Flag Cnt': 'FIN Flag Count',
    'SYN Flag Count': 'SYN Flag Count',
    'SYN Flag Cnt': 'SYN Flag Count',
    'RST Flag Count': 'RST Flag Count',
    'RST Flag Cnt': 'RST Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'PSH Flag Cnt': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'ACK Flag Cnt': 'ACK Flag Count',
    'URG Flag Count': 'URG Flag Count',
    'URG Flag Cnt': 'URG Flag Count',
    'CWE Flag Count': 'CWE Flag Count',
    'CWE Flag Cnt': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Count',
    'ECE Flag Cnt': 'ECE Flag Count',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Average Packet Size': 'Avg Packet Size',
    'Pkt Size Avg': 'Avg Packet Size',
    'Avg Fwd Segment Size': 'Avg Fwd Segment Size',
    'Fwd Seg Size Avg': 'Avg Fwd Segment Size',
    'Avg Bwd Segment Size': 'Avg Bwd Segment Size',
    'Bwd Seg Size Avg': 'Avg Bwd Segment Size',
    'Fwd Avg Bytes/Bulk': 'Fwd Avg Bytes/Bulk',
    'Fwd Byts/b Avg': 'Fwd Avg Bytes/Bulk',
    'Fwd Avg Packets/Bulk': 'Fwd Avg Packets/Bulk',
    'Fwd Pkts/b Avg': 'Fwd Avg Packets/Bulk',
    'Fwd Avg Bulk Rate': 'Fwd Avg Bulk Rate',
    'Fwd Blk Rate Avg': 'Fwd Avg Bulk Rate',
    'Bwd Avg Bytes/Bulk': 'Bwd Avg Bytes/Bulk',
    'Bwd Byts/b Avg': 'Bwd Avg Bytes/Bulk',
    'Bwd Avg Packets/Bulk': 'Bwd Avg Packets/Bulk',
    'Bwd Pkts/b Avg': 'Bwd Avg Packets/Bulk',
    'Bwd Avg Bulk Rate': 'Bwd Avg Bulk Rate',
    'Bwd Blk Rate Avg': 'Bwd Avg Bulk Rate',
    'Subflow Fwd Packets': 'Subflow Fwd Packets',
    'Subflow Fwd Pkts': 'Subflow Fwd Packets',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Subflow Fwd Byts': 'Subflow Fwd Bytes',
    'Subflow Bwd Packets': 'Subflow Bwd Packets',
    'Subflow Bwd Pkts': 'Subflow Bwd Packets',
    'Subflow Bwd Bytes': 'Subflow Bwd Bytes',
    'Subflow Bwd Byts': 'Subflow Bwd Bytes',
    'Init_Win_bytes_forward': 'Init Fwd Win Bytes',
    'Init Fwd Win Byts': 'Init Fwd Win Bytes',
    'Init_Win_bytes_backward': 'Init Bwd Win Bytes',
    'Init Bwd Win Byts': 'Init Bwd Win Bytes',
    'act_data_pkt_fwd': 'Fwd Act Data Packets',
    'Fwd Act Data Pkts': 'Fwd Act Data Packets',
    'min_seg_size_forward': 'Fwd Seg Size Min',
    'Fwd Seg Size Min': 'Fwd Seg Size Min',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min',
    'Label': 'Label'
}

In [5]:
def readHugeCsvFileAsDataFrame(file_path):
    # start = timer()
    dtypes = {'SimillarHTTP': 'object',
              'Timestamp': 'object',
              'Source IP': 'str',
              'Destination IP': 'str',
              'Flow ID': 'object',
              'Label': 'object',
              }
    for feature in [f'f_{i}' for i in range(82)]:
        dtypes[feature] = "float32"

    dask_df = dd.read_csv(file_path, low_memory=False,blocksize=50000, dtype=dtypes)  # 50MB chunk-size
    # elapsed_time = timer() - start
    # print("Read csv with dask: ", elapsed_time, "sec")
    return dask_df.compute()

With Scheduler (see the elapsed time to process)

In [6]:
%%time
scheduler = Client()
df = scheduler.submit(readHugeCsvFileAsDataFrame, path_UDPLag)
workingDataFrame = df.result()
workingDataFrame

Wall time: 2min 27s


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,186059,172.16.0.5-192.168.50.1-58445-4463-17,172.16.0.5,58445,192.168.50.1,4463,17,2018-12-01 13:04:45.928673,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
1,135692,172.16.0.5-192.168.50.1-36908-9914-17,172.16.0.5,36908,192.168.50.1,9914,17,2018-12-01 13:04:45.928913,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
2,33822,172.16.0.5-192.168.50.1-41727-32361-17,172.16.0.5,41727,192.168.50.1,32361,17,2018-12-01 13:04:45.928915,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
3,24498,172.16.0.5-192.168.50.1-55447-5691-17,172.16.0.5,55447,192.168.50.1,5691,17,2018-12-01 13:04:45.929024,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
4,117372,172.16.0.5-192.168.50.1-58794-56335-17,172.16.0.5,58794,192.168.50.1,56335,17,2018-12-01 13:04:45.929096,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,350624,172.16.0.5-192.168.50.1-60490-14102-6,172.16.0.5,60490,192.168.50.1,14102,6,2018-12-01 13:30:30.740273,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
123,336856,172.16.0.5-192.168.50.1-60491-58360-6,172.16.0.5,60491,192.168.50.1,58360,6,2018-12-01 13:30:30.740323,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
124,115128,172.16.0.5-192.168.50.1-60492-2905-6,172.16.0.5,60492,192.168.50.1,2905,6,2018-12-01 13:30:30.740374,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
125,51370,172.16.0.5-192.168.50.1-60493-45714-6,172.16.0.5,60493,192.168.50.1,45714,6,2018-12-01 13:30:30.740424,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag


In [7]:
mb = workingDataFrame.memory_usage().sum() / 1024**2
print('Memory usage of Working Dataframe: {:.2f} MB'.format(mb))

Memory usage of dataframe is 251.65 MB


In [86]:
len(list(workingDataFrame.columns))

88

In [87]:
workingDataFrame.nunique(axis=0)

Unnamed: 0         370605
Flow ID            368680
 Source IP            117
 Source Port        60938
 Destination IP       138
                    ...  
 Idle Max           62358
 Idle Min           62272
SimillarHTTP           89
 Inbound                2
 Label                  3
Length: 88, dtype: int64

Following Columns may have little insignificance over model creation. Follow Below Ling
https://link.springer.com/chapter/10.1007/978-3-031-09484-2_2

In [2]:
# src_port,dst_port, src_ip, dst_ip these are duplicate
drop_columns = [
    "Flow ID",
    'Fwd Header Length.1',
    "Source IP", "Src IP",
    "Source Port", "Src Port",
    "Destination IP", "Dst IP",
    "Destination Port", "Dst Port",
    "Timestamp",
    "Unnamed: 0",
    "Inbound",
    "SimillarHTTP"
]

Dropping Unnecessary Features & maintain column consistency

In [67]:
workingDataFrame.columns = workingDataFrame.columns.str.strip()  # sometimes there's leading / trailing whitespace
workingDataFrame.rename(columns=col_name_consistency, inplace=True)
workingDataFrame.drop(columns=drop_columns, inplace=True, errors='ignore')

workingDataFrame.head()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,58445,4463,17,1,2,0,766.0,0.0,383.0,383.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
1,36908,9914,17,1,2,0,778.0,0.0,389.0,389.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
2,41727,32361,17,2,2,0,750.0,0.0,375.0,375.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
3,55447,5691,17,2,2,0,738.0,0.0,369.0,369.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
4,58794,56335,17,1,2,0,750.0,0.0,375.0,375.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag


Data Cleaning Based on Data Types (DownSizing)

In [68]:
# workingDataFrame.dtypes
workingDataFrame.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370605 entries, 0 to 126
Data columns (total 80 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Source Port               370605 non-null  int64  
 1   Destination Port          370605 non-null  int64  
 2   Protocol                  370605 non-null  int64  
 3   Flow Duration             370605 non-null  int64  
 4   Total Fwd Packets         370605 non-null  int64  
 5   Total Backward Packets    370605 non-null  int64  
 6   Fwd Packets Length Total  370605 non-null  float64
 7   Bwd Packets Length Total  370605 non-null  float64
 8   Fwd Packet Length Max     370605 non-null  float64
 9   Fwd Packet Length Min     370605 non-null  float64
 10  Fwd Packet Length Mean    370605 non-null  float64
 11  Fwd Packet Length Std     370605 non-null  float64
 12  Bwd Packet Length Max     370605 non-null  float64
 13  Bwd Packet Length Min     370605 non-null  floa

In [69]:
# converting 64bit float to 32 bit float & 64bit integer to 8bit integer
for column in workingDataFrame:
    if workingDataFrame[column].dtype == 'float64':
        workingDataFrame[column] = pd.to_numeric(workingDataFrame[column], downcast='float')
    if workingDataFrame[column].dtype == 'int64':
        workingDataFrame[column] = pd.to_numeric(workingDataFrame[column], downcast='integer')

workingDataFrame.info(memory_usage="deep") # observe the dTypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370605 entries, 0 to 126
Data columns (total 80 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Source Port               370605 non-null  int32  
 1   Destination Port          370605 non-null  int32  
 2   Protocol                  370605 non-null  int8   
 3   Flow Duration             370605 non-null  int32  
 4   Total Fwd Packets         370605 non-null  int16  
 5   Total Backward Packets    370605 non-null  int16  
 6   Fwd Packets Length Total  370605 non-null  float32
 7   Bwd Packets Length Total  370605 non-null  float32
 8   Fwd Packet Length Max     370605 non-null  float32
 9   Fwd Packet Length Min     370605 non-null  float32
 10  Fwd Packet Length Mean    370605 non-null  float32
 11  Fwd Packet Length Std     370605 non-null  float32
 12  Bwd Packet Length Max     370605 non-null  float32
 13  Bwd Packet Length Min     370605 non-null  floa

Finding number of missing values contains features in the dataset

In [70]:
workingDataFrame.isnull().sum()

Source Port          0
Destination Port     0
Protocol             0
Flow Duration        0
Total Fwd Packets    0
                    ..
Idle Mean            0
Idle Std             0
Idle Max             0
Idle Min             0
Label                0
Length: 80, dtype: int64

Removing NaN values

In [71]:
print(f"pure missing value contains rows: {workingDataFrame.isna().any(axis=1).sum()}")
# workingDataFrame.isna().sum()

pure missing value contains rows: 36132


plus infinity or negative infinity columns also made as nan, to remove all easily

In [72]:
workingDataFrame.replace([np.inf, -np.inf], np.nan, inplace=True)
print(f"missing value contains total rows: {workingDataFrame.isna().any(axis=1).sum()}")

missing value contains total rows: 36403


After Dropping NaN values, want to see total rows.

In [73]:
workingDataFrame.dropna(inplace=True)
print(f"After dropping NaN values, number of rows: {workingDataFrame.shape[0]}")
print(f"After dropping NaN values, number of columns: {workingDataFrame.shape[1]}")

After dropping NaN values, number of rows: 334202
After dropping NaN values, number of columns: 80


That's why, Converting pandas dataFrame into dask's dataFrame

ddf = dd.from_pandas(workingDataFrame, npartitions=10)
ddf.compute()

Dropping Duplicates
There should be no duplicates because they can bias training and can lead to over-optimistic estimates of classification performance during testing.

In [74]:
workingDataFrame.duplicated().sum()

49

Fully duplicate rows to be removed

In [75]:
workingDataFrame.drop_duplicates(inplace=True)
workingDataFrame.reset_index(inplace=True, drop=True)
print(f"After dropping NaN values, number of rows: {workingDataFrame.shape[0]}")
print(f"After dropping NaN values, number of columns: {workingDataFrame.shape[1]}")
print(f"After dropping NaN values, unique no of Source IP: {workingDataFrame['Source Port'].nunique()}")
print(f"After dropping NaN values, unique no of Destination IP: {workingDataFrame['Destination Port'].nunique()}")
# print(workingDataFrame["Source IP"].unique())
# print(workingDataFrame["Destination IP"].unique())

After dropping NaN values, number of rows: 334153
After dropping NaN values, number of columns: 80
After dropping NaN values, unique no of Source IP: 60937
After dropping NaN values, unique no of Destination IP: 65088


After data cleaning, how much storage it holds now

In [ ]:
workingDataFrame.info(memory_usage="deep")

Now arising problem is that, row number has been significantly reduced but feature numbers are still 80!!
So, need feature engineering here

In [ ]:
columnList = workingDataFrame.columns.tolist()
# columnList[:-1]  # just for now, Omitting 'label' column

from sklearn.feature_selection import VarianceThreshold

var_thr = VarianceThreshold(threshold=0.1)  #Removing both constant and quasi-constant
var_thr.fit(workingDataFrame[columnList[:-1]])

var_threshold_bool_list = var_thr.get_support()
var_threshold_bool_list_after_label_added = np.append(var_threshold_bool_list,True)
var_threshold_bool_list_after_label_added

In [ ]:
concol = [column for column in workingDataFrame.columns
          if column not in workingDataFrame.columns[var_threshold_bool_list_after_label_added]]

for omittedFeatures in concol:
    print(omittedFeatures)

workingDataFrame.drop(columns=concol, inplace=True, errors='ignore')
workingDataFrame

In [76]:

def featueEngineeringBasedOnZero(dataFrameArg, thresholdPercentage, showPercentage):
    totalCols = dataFrameArg.shape[1]
    totalRows = len(dataFrameArg)
    unNecessaryFeatureCount = 0
    unNecessaryFeatureNames = []

    for column in dataFrameArg:
        zerosInCol = (dataFrameArg[column] == 0).sum()
        if zerosInCol != 0:
            percentageOfZerosInRow = ((zerosInCol * 100) / totalRows)

            if showPercentage:
                print(column, "\t\t-\t\t", zerosInCol, "\t\t-\t\t", percentageOfZerosInRow)

            if percentageOfZerosInRow > thresholdPercentage:
                unNecessaryFeatureNames.append(column)
                unNecessaryFeatureCount = unNecessaryFeatureCount + 1

    print("\nTotal features having more than ", thresholdPercentage, "% zero are - ", unNecessaryFeatureCount,
          "out of ",
          totalCols)
    return unNecessaryFeatureNames

Identifying those features containing 99% zeroes

In [77]:
featureContainingAlmostZero = featueEngineeringBasedOnZero(dataFrameArg=workingDataFrame, thresholdPercentage=99,showPercentage=False)
featureContainingAlmostZero


Total features having more than  99 % zero are -  25 out of  80


['Bwd Packets Length Total',
 'Bwd Packet Length Max',
 'Bwd Packet Length Min',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Bwd IAT Std',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'FIN Flag Count',
 'SYN Flag Count',
 'RST Flag Count',
 'PSH Flag Count',
 'URG Flag Count',
 'CWE Flag Count',
 'ECE Flag Count',
 'Avg Bwd Segment Size',
 'Fwd Avg Bytes/Bulk',
 'Fwd Avg Packets/Bulk',
 'Fwd Avg Bulk Rate',
 'Bwd Avg Bytes/Bulk',
 'Bwd Avg Packets/Bulk',
 'Bwd Avg Bulk Rate',
 'Subflow Bwd Bytes']

Omitting above features containing 99% zeroes

In [78]:
workingDataFrame.drop(columns=featureContainingAlmostZero, inplace=True, errors='ignore')
workingDataFrame.rename(columns=col_name_consistency, inplace=True)
workingDataFrame.reset_index(inplace=True, drop=True)
workingDataFrame

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,58445,4463,17,1,2,0,766.0,383.0,383.0,383.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
1,36908,9914,17,1,2,0,778.0,389.0,389.0,389.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
2,41727,32361,17,2,2,0,750.0,375.0,375.0,375.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
3,55447,5691,17,2,2,0,738.0,369.0,369.0,369.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
4,58794,56335,17,1,2,0,750.0,375.0,375.0,375.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334148,60489,27808,6,1,2,0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
334149,60490,14102,6,1,2,0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
334150,60491,58360,6,1,2,0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag
334151,60492,2905,6,2,2,0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP-lag


Now, To see the number of unique values in each column

In [79]:
workingDataFrame.nunique(axis=0)

Source Port                 60937
Destination Port            65088
Protocol                        3
Flow Duration               77044
Total Fwd Packets             102
Total Backward Packets        111
Fwd Packets Length Total      648
Fwd Packet Length Max         448
Fwd Packet Length Min         126
Fwd Packet Length Mean        724
Fwd Packet Length Std         632
Flow Bytes/s                15607
Flow Packets/s              77138
Flow IAT Mean               77156
Flow IAT Std                83362
Flow IAT Max                73387
Flow IAT Min                  340
Fwd IAT Total               75510
Fwd IAT Mean                75594
Fwd IAT Std                 80829
Fwd IAT Max                 72086
Fwd IAT Min                   141
Bwd IAT Total                3300
Bwd IAT Mean                 3317
Bwd IAT Max                  3283
Bwd IAT Min                    87
Fwd Header Length             331
Bwd Header Length             204
Fwd Packets/s               77062
Bwd Packets/s 

In [ ]:
workingDataFrame

Profiling of all features to find out the co-relations

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(workingDataFrame, title="DDOS")
profile.to_file("UdpLag.html")

Saving New DataFrame as csv file to new location

In [ ]:
def dataCleaningResultToAnotherCSV(dataFrameArg, dirPath, file_name):
    dataFrameArg.to_csv(dirPath + file_name)

In [ ]:
newCsvPath = "D:\\thesis_dataset\\01-12\\AfterDataCleaning(Final)\\"
newFileName = "UDPLag.csv"
# newFileName = "Syn.csv"
# newFileName = "DrDoS_NTP.csv"
# newFileName = "DrDoS_LDAP.csv"
# newFileName = "DrDoS_SSDP.csv"
# newFileName = "DrDoS_UDP.csv"
dataCleaningResultToAnotherCSV(dataFrameArg=workingDataFrame, dirPath=newCsvPath, file_name=newFileName)

In [ ]:
import gc
gc.collect()
